# Mockup Version 1

- Mock-up the workflow from MIS global summit in an environment like EvidenceCare's Epic App. Due January 1.
    - ☑️ List content for our app (Due by Nov 23) TJM
    - ☑️ Define the workflow between user, website, and service (Due by Nov 23) TJM & NS workshop session
    - Define interface between UI and service (Due by Nov 30) TJM & NS workshop session
    - Make a webpage that looks like an Epic screen (Due by Dec 7) TJM
    - Flask configuration (due by Dec 1) TJM (Nikesh's static IP: 122:169:102:165) - in router, 
    - Port configuration (due by Dec 1) TJM & NS
    - Write & test the code (Due by Dec 23) TJM & NS
    - Provide instructions, documentation, supporting information to make it (Due by Dec 23) BED

Define and test functions in jupyter notebook
- ☑️ delete UMLS concepts from database
- ☑️ re-import UMLS concepts into database using the improved data model
- ☑️ get a CSV of common problems (at least 400 instances each), with the corresponding CUI  
- modify existing functions for the new data model
    - ☑️ comorbidities_of(prob_list)
    - prob_assoc_abnl_lab(cui_prob_list)
    - Rx_assoc_with_prob_CUI(prob_CUI)
- Save functions to separate python scripts
- Send Nikesh the comorbidities_of_CUI(cui_prob_list) function script, IP, database login info
- Write a dummy function that returns assessment and plan in JSON format

In [68]:
import pandas as pd
import time

In [3]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()


Please enter the Neo4j database password to continue 
 ···············


In [9]:
# get a CSV of all problems in the database that have enough data to potentially return robust results
query = '''
MATCH (p:Problem)-[:INSTANCE_OF]->(c:Concept)
WITH c.cui AS CUIs, count(p) as instances
WHERE instances > 400
MATCH (c2:Concept)
WHERE c2.cui_pref_term IS NOT NULL AND c2.cui IN CUIs
RETURN c2.term as problem, c2.cui as CUI'''
data = session.run(query)

# Convert the neo4j object into a dataframe
common_problems_df = pd.DataFrame([dict(record) for record in data])

In [55]:
common_problems_df.sort_values(by='problem', inplace=True)
common_problems_df

NameError: name 'common_problems_df' is not defined

In [14]:
# Write out to CSV
common_problems_df.to_csv('common_problems.csv', index=False, encoding='utf-8')

## Modify existing functions for the new data model

In [62]:
cui_prob_list = ['C3714660', 'C0085762']

    query = '''
    MATCH (prob1:Problem)<-[:HAD_PROBLEM]-(pt:Patients)
    WHERE prob1.cui in {cui_prob_list}
    WITH distinct(pt) AS patients
    MATCH (patients)-[:HAD_PROBLEM]->(prob2:Problem)
    WITH prob2.cui AS CUIs, count(prob2) AS Number
    MATCH (c:Concept)
    WHERE c.cui IN CUIs AND c.cui_pref_term IS NOT NULL 
    RETURN c.term as Potential_Problem, c.cui AS CUI, Number
    ORDER BY Number DESC
    '''.format(cui_prob_list=cui_prob_list)
    comorbidities = session.run(query)
    comorbidities = pd.DataFrame([dict(record) for record in comorbidities])
    
    query = '''
    MATCH (excluded:Problem)
    WHERE excluded.cui in {cui_prob_list}
    WITH collect(excluded) as excluded
    MATCH (pt:Patients)-[:HAD_PROBLEM]->(prob:Problem)
    WITH excluded, pt, collect(prob) as problems
    WHERE NONE (prob in problems where prob in excluded)
    MATCH (pt)-[:HAD_PROBLEM]-(prob2:Problem)
    WITH prob2.cui AS CUIs, count(prob2) AS Number
    MATCH (c:Concept)
    WHERE c.cui IN CUIs AND c.cui_pref_term IS NOT NULL 
    RETURN c.term as Potential_Problem, c.cui AS CUI, Number
    ORDER BY Number DESC
    '''.format(cui_prob_list=cui_prob_list)
    gen_problems = session.run(query)
    gen_problems = pd.DataFrame([dict(record) for record in gen_problems])
    
    gen_pop_total = sum(gen_problems['Number'])
    gen_problems['Gen_pop_proportion'] = gen_problems['Number']/gen_pop_total
    
    gen_problems = gen_problems[gen_problems['Number'] > 50]
    
    comorb_total = sum(comorbidities['Number'])
    comorbidities['Comorbidities_proportion'] = comorbidities['Number']/comorb_total
    
    comorbidities = comorbidities[comorbidities['Number'] > 200/len(cui_prob_list)]
    
    # Merge the "Gen_pop_proportion" column from gen_problems into comorbidities
    comorbidities = pd.merge(comorbidities, gen_problems, on=['CUI', 'Potential_Problem'])
    
    comorbidities.head()
    
    comorbidities['Odds_Ratio'] = (comorbidities['Comorbidities_proportion']/comorbidities['Gen_pop_proportion'])
    comorbidities.sort_values(by='Odds_Ratio', ascending=False, inplace=True)
    
    comorbidities.loc[:,['Potential_Problem', 'Odds_Ratio']].head(20)

,Description,Odds_Ratio
5,"Liver Cirrhosis, Alcoholic",18.020708
1,Mediastinal Emphysema,17.820327
3,Alcohol withdrawal syndrome,13.115391
0,Agitation,9.292636
6,Seizures,3.291711
2,Liver Failure,2.071306
4,"Diabetes Mellitus, Non-Insulin-Dependent",1.144112


In [109]:
def comorbidities_of_CUI(cui_prob_list):
    
    query = '''
    MATCH (prob1:Problem)<-[:HAD_PROBLEM]-(pt:Patients)
    WHERE prob1.cui in {cui_prob_list}
    WITH distinct(pt) AS patients
    MATCH (patients)-[:HAD_PROBLEM]->(prob2:Problem)
    WITH prob2.cui AS CUIs, count(prob2) AS Number
    MATCH (c:Concept)
    WHERE c.cui IN CUIs AND c.cui_pref_term IS NOT NULL 
    RETURN c.term as Potential_Problem, c.cui AS CUI, Number
    ORDER BY Number DESC
    '''.format(cui_prob_list=cui_prob_list)
    comorbidities = session.run(query)
    comorbidities = pd.DataFrame([dict(record) for record in comorbidities])
    
    query = '''
    MATCH (excluded:Problem)
    WHERE excluded.cui in {cui_prob_list}
    WITH collect(excluded) as excluded
    MATCH (pt:Patients)-[:HAD_PROBLEM]->(prob:Problem)
    WITH excluded, pt, collect(prob) as problems
    WHERE NONE (prob in problems where prob in excluded)
    MATCH (pt)-[:HAD_PROBLEM]-(prob2:Problem)
    WITH prob2.cui AS CUIs, count(prob2) AS Number
    MATCH (c:Concept)
    WHERE c.cui IN CUIs AND c.cui_pref_term IS NOT NULL 
    RETURN c.term as Potential_Problem, c.cui AS CUI, Number
    ORDER BY Number DESC
    '''.format(cui_prob_list=cui_prob_list)
    gen_problems = session.run(query)
    gen_problems = pd.DataFrame([dict(record) for record in gen_problems])
    
    gen_pop_total = sum(gen_problems['Number'])
    gen_problems['Gen_pop_proportion'] = gen_problems['Number']/gen_pop_total
    
    gen_problems = gen_problems[gen_problems['Number'] > 25]
    
    comorb_total = sum(comorbidities['Number'])
    comorbidities['Comorbidities_proportion'] = comorbidities['Number']/comorb_total
    
    comorbidities = comorbidities[comorbidities['Number'] > 75/len(cui_prob_list)]
    
    # Merge the "Gen_pop_proportion" column from gen_problems into comorbidities
    comorbidities = pd.merge(comorbidities, gen_problems, on=['CUI', 'Potential_Problem'])
    
    comorbidities.head()
    
    comorbidities['Odds_Ratio'] = (comorbidities['Comorbidities_proportion']/comorbidities['Gen_pop_proportion'])
    comorbidities.sort_values(by='Odds_Ratio', ascending=False, inplace=True)
    
    return comorbidities.loc[:,['Potential_Problem', 'Odds_Ratio']].head(10)
#     return comorbidities.head(10)

In [112]:
start_time = time.time()

cui_prob_list = ['C1565489', 'C0085762']
result_df = comorbidities_of_CUI(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")
result_df

Total runtime: 0.8025097846984863 seconds


,Potential_Problem,Odds_Ratio
5,Spinal Fractures,33.967289
58,Adenoma,16.911680
72,Foot Ulcer,15.174249
68,Septicemia,14.704117
18,Respiratory arrest,14.635917
10,"Liver Cirrhosis, Alcoholic",14.437897
6,Alcohol withdrawal syndrome,10.507838
46,Peripheral Neuropathy,10.254955
2,Gallstone pancreatitis,10.007094
13,"Hepatitis, Alcoholic",9.941156


In [107]:
jsonify(result_df)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [108]:
def comorbidities_of_CUI(cui_prob_list):
    
    query = '''
    MATCH (prob1:Problem)<-[:HAD_PROBLEM]-(pt:Patients)
    WHERE prob1.cui in {cui_prob_list}
    WITH distinct(pt) AS patients
    MATCH (patients)-[:HAD_PROBLEM]->(prob2:Problem)
    WITH prob2.cui AS CUIs, count(prob2) AS Number
    MATCH (c:Concept)
    WHERE c.cui IN CUIs AND c.cui_pref_term IS NOT NULL 
    RETURN c.term as Potential_Problem, c.cui AS CUI, Number
    ORDER BY Number DESC
    '''.format(cui_prob_list=cui_prob_list)
    comorbidities = session.run(query)
    comorbidities = pd.DataFrame([dict(record) for record in comorbidities])
    
    query = '''
    MATCH (excluded:Problem)
    WHERE excluded.cui in {cui_prob_list}
    WITH collect(excluded) as excluded
    MATCH (pt:Patients)-[:HAD_PROBLEM]->(prob:Problem)
    WITH excluded, pt, collect(prob) as problems
    WHERE NONE (prob in problems where prob in excluded)
    MATCH (pt)-[:HAD_PROBLEM]-(prob2:Problem)
    WITH prob2.cui AS CUIs, count(prob2) AS Number
    MATCH (c:Concept)
    WHERE c.cui IN CUIs AND c.cui_pref_term IS NOT NULL 
    RETURN c.term as Potential_Problem, c.cui AS CUI, Number
    ORDER BY Number DESC
    '''.format(cui_prob_list=cui_prob_list)
    gen_problems = session.run(query)
    gen_problems = pd.DataFrame([dict(record) for record in gen_problems])
    
    gen_pop_total = sum(gen_problems['Number'])
    gen_problems['Gen_pop_proportion'] = gen_problems['Number']/gen_pop_total
    
    gen_problems = gen_problems[gen_problems['Number'] > 25]
    
    comorb_total = sum(comorbidities['Number'])
    comorbidities['Comorbidities_proportion'] = comorbidities['Number']/comorb_total
    
    comorbidities = comorbidities[comorbidities['Number'] > 75/len(cui_prob_list)]
    
    # Merge the "Gen_pop_proportion" column from gen_problems into comorbidities
    comorbidities = pd.merge(comorbidities, gen_problems, on=['CUI', 'Potential_Problem'])
    
    comorbidities.head()
    
    comorbidities['Odds_Ratio'] = (comorbidities['Comorbidities_proportion']/comorbidities['Gen_pop_proportion'])
    comorbidities.sort_values(by='Odds_Ratio', ascending=False, inplace=True)
    
    return comorbidities.loc[:,['Potential_Problem', 'Odds_Ratio']].head(10)

In [93]:
import flask
from flask import request, jsonify


# Create some test data for our catalog in the form of a list of dictionaries.
books = [
    {'id': 0,
     'title': 'A Fire Upon the Deep',
     'author': 'Vernor Vinge',
     'img_url':'https://upload.wikimedia.org/wikipedia/en/4/4a/A_Fire_Upon_the_Deep.bookcover.jpg',
     'first_sentence': 'The coldsleep itself was dreamless.',
     'year_published': '1992'},
    {'id': 1,
     'title': 'The Ones Who Walk Away From Omelas',
     'author': 'Ursula K. Le Guin',
     'img_url':'https://m.media-amazon.com/images/I/41srw9ZyJrL.jpg',
     'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.',
     'published': '1973'},
    {'id': 2,
     'title': 'Dhalgren',
     'author': 'Samuel R. Delany',
     'img_url':'https://upload.wikimedia.org/wikipedia/en/thumb/a/ac/Dhalgren-bantam-cover.jpg/220px-Dhalgren-bantam-cover.jpg',
     'first_sentence': 'to wound the autumnal city.',
     'published': '1975'}
]


@app.route('/', methods=['GET'])
def home():
    return '''<h1>Distant Reading Archive</h1>
<p>A prototype API for distant reading of science fiction novels.</p>'''


@app.route('/api/v1/resources/books/all', methods=['GET'])
def api_all():
    return jsonify(books)


@app.route('/api/v1/resources/books', methods=['GET'])
def api_id():
    # Check if an ID was provided as part of the URL.
    # If ID is provided, assign it to a variable.
    # If no ID is provided, display an error in the browser.
    if 'id' in request.args:
        id = int(request.args['id'])
    else:
        return "Error: No id field provided. Please specify an id."

    # Create an empty list for our results
    results = []

    # Loop through the data and match results that fit the requested ID.
    # IDs are unique, but other fields might return many results
    for book in books:
        if book['id'] == id:
            results.append(book)

    # Use the jsonify function from Flask to convert our list of
    # Python dictionaries to the JSON format.
    return jsonify(results)